In [ ]:
!pip install -U rank-bm25 openai
from google.colab import drive
drive.mount('/content/gdrive')
!cp /content/gdrive/MyDrive/CSI6900/pre_processing.py /content

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.0 MB/s eta 0:00:00
Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import random
from ast import literal_eval
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from rank_bm25 import BM25Okapi
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import torch

nltk.download('punkt')
nltk.download('stopwords')

# Run locally
#DATA_PATH = ''

# colab
DATA_PATH = '/content/gdrive/MyDrive/CSI6900/'

device = torch.device("mps" if getattr(torch,'has_mps',False) else "cuda:0" if torch.cuda.is_available() else "cpu")
device

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


device(type='cpu')

## Load data

In [ ]:
infoSheet = pd.read_csv(DATA_PATH + 'data/openai_embedding/infoSheet_name_abstract.csv')
infoSheet['embedding'] = infoSheet['embedding'].apply(literal_eval)
print(infoSheet.isnull().sum())
print('Length: ', len(infoSheet.index))
infoSheet.head(10)

ID                       0
name_en                  0
name_fr                 87
abstract_en              0
abstract_fr             89
description_en           0
description_fr          87
taxonomy heading ids     0
embedding                0
dtype: int64
Length:  230


,ID,name_en,name_fr,abstract_en,abstract_fr,description_en,description_fr,taxonomy heading ids,embedding
0,8920,Abuse and Domestic Violence,Maltraitance et violence familiale,"Abuse is behaviour used to intimidate, isolate...",La maltraitance est un comportement visant à i...,\r\n\tWhat is Abuse and Domestic Violence?\r\n...,\r\n\tQu&#39;est-ce que la maltraitance et la ...,"0,219,58,8,7,6,50,9,36,54,37","[-0.006836807355284691, -1.2969143426744267e-0..."
1,69660,"ADHD in Children, Youth and Adults: Informatio...",NaN,Attention deficit hyperactivity disorder (ADHD...,NaN,"\r\n\tAbbreviations\r\n\r\n\tADHD, attention-d...",NaN,"0,13","[6.297678191913292e-05, 0.005104047246277332, ..."
2,20505,ADHD: Managing ADHD Medications,TDAH: Gestion des médicaments pour le TDAH,Although ADHD medications may be helpful for s...,Bien que les médicaments du TDAH peuvent être ...,\r\n\tGeneral Considerations\r\n\r\n\t\r\n\t\t...,\r\n\tObservations générales\r\n\r\n\t\r\n\t\t...,"0,13","[-0.007076850160956383, -0.0020307896193116903..."
3,8917,Alcohol and Substance Use Problems in Children...,Problèmes de l’alcoolisme et la toxicomanie ch...,Many children/youth experiment with alcohol an...,Nombreux sont les enfants et les jeunes qui fo...,\r\n\tDavid's Story\r\n\r\n\tUp until this sch...,\r\n\tL’histoire de David\r\n\r\n\tJusqu’à cet...,"0,12","[-0.0038117102812975645, -0.008029664866626263..."
4,8909,Alcohol: Cutting Back or Quitting Drinking,Problèmes liés à la consommation d'alcool,Drinking alcohol is an accepted practice in ma...,La consommation d'alcool est une pratique acce...,\r\n\tIntroduction\r\n\r\n\tMany people who dr...,\r\n\tIntroduction\r\n\r\n\tNombreuses sont le...,"0,12","[-0.010912169702351093, 0.0006169816479086876,..."
5,61003,Alcohol Use Disorder in Adults: Information fo...,NaN,Alcohol use problems are common in primary car...,NaN,\r\n\tEpidemiology\r\n\r\n\tPrevalence\r\n\r\n...,NaN,"0,12,266","[-0.0002503542636986822, 0.0016840933822095394..."
6,18393,Alcohol Use Disorder in Youth: Information for...,NaN,Many youth will experiment with alcohol and su...,NaN,Epidemiology\r\n\r\n\t\r\n\t\t10% of the&nbsp;...,NaN,"0,12,266","[-0.007133564446121454, -0.0029860695358365774..."
7,24390,Alzheimer's Dementia: Information for Primary ...,NaN,Alzheimer’s is the most common form of dementi...,NaN,\r\n\tCase\r\n\r\n\t\r\n\t\tYour 80 year old p...,NaN,"0,79","[-0.0032352525740861893, 0.0039140209555625916..."
8,11564,Anger,La colère,Anger is a normal emotion that we get when we ...,La colère est une émotion normale que nous res...,\r\n\tDoes any of this sound like you?\r\n\r\n...,\r\n\tEst-ce que cela trouve un &eacute;cho ch...,"0,14,50,54,9,36,37","[-0.0030424355063587427, 0.006882417947053909,..."
9,26975,Anorexia Nervosa: Information for Primary Care,NaN,Anorexia nervosa is a common eating disorder a...,NaN,\r\n\tCase\r\nIdentifying data&nbsp;32-yo fema...,NaN,"0,22","[-3.332154960844491e-07, -0.000195899716345593..."


In [ ]:
taxonomy = pd.read_csv(DATA_PATH + 'data/openai_embedding/taxonomy_name_description.csv')
taxonomy['embedding'] = taxonomy['embedding'].apply(literal_eval)
# Drop rows where 'abstract_en' is NaN
taxonomy.dropna(subset=['description'], inplace=True)
taxonomy.reset_index(drop=True, inplace=True)
print(taxonomy.isnull().sum())
print('Length: ', len(taxonomy.index))
taxonomy.head(10)

id              0
name            0
description     0
translations    0
embedding       0
dtype: int64
Length:  192


,id,name,description,translations,embedding
0,1,Root,Root,"{""name"":{""en"":""Root"",""fr"":null},""description"":...","[-0.01242552325129509, 0.0049454341642558575, ..."
1,2,All Mental Health Resources,The listings of mental health resources includ...,"{""name"":{""en"":""All Mental Health Resources"",""f...","[-0.003916683606803417, 0.0043426863849163055,..."
2,3,Crisis and Emergency,Refers to all programs that provide immediate ...,"{""name"":{""en"":""Crisis and Emergency"",""fr"":""Res...","[-0.0016540151555091143, 0.004231865983456373,..."
3,4,"System Navigation, including Information and R...","Are you looking for help, but dont know where ...","{""name"":{""en"":""System Navigation, including In...","[-0.0052290805615484715, 0.001998607302084565,..."
4,5,Child Welfare including Children's Aid Society...,The child welfare / child protection system is...,"{""name"":{""en"":""Child Welfare including Childre...","[-0.0017937872326001525, 0.002536701038479805,..."
5,6,Emergency Shelter and Housing,There are various shelters that people can use...,"{""name"":{""en"":""Emergency Shelter and Housing"",...","[-0.003927853424102068, 0.005480578169226646, ..."
6,7,Hospital Emergency Department,Is there an emergency such as medical or menta...,"{""name"":{""en"":""Hospital Emergency Department"",...","[-0.01046136673539877, 0.001888505183160305, -..."
7,8,"Crisis Lines including Telephone, Online and Chat","Are you in a crisis? Crisis lines offer free, ...","{""name"":{""en"":""Crisis Lines including Telephon...","[-0.006740534212440252, -0.004541291855275631,..."
8,9,Psychiatrists,Psychiatrists are medical doctors who have rec...,"{""name"":{""en"":""Psychiatrists"",""fr"":""Psychiatre...","[0.00967669952660799, 0.0008635374833829701, -..."
9,10,A-Z Mental Health Conditions and Topics,Alphabetical list of mental health topics and ...,"{""name"":{""en"":""A-Z Mental Health Conditions an...","[-0.010088824667036533, 0.0018147194059565663,..."


## Pre-processing

In [ ]:
# Remove stop words and stemming
stop_words = set(stopwords.words('english'))
def preprocessing(sentence):
    stemmer = PorterStemmer()
    text_tokens = word_tokenize(sentence)
    tokens_without_sw = [word for word in text_tokens if not word.lower() in stop_words]
    tokens_stemmed = [stemmer.stem(word) for word in tokens_without_sw]
    return tokens_stemmed

In [ ]:
preprocessing('Machine Learning is Cool!')

['machin', 'learn', 'cool', '!']

## BM25 Search

Select top 100 results

In [ ]:
# Randomly choose 10 infoSheets
search_term_indices = [random.randint(0, len(infoSheet.index)) for i in range(10)]
search_term_indices = [113, 10, 124, 39, 88, 23, 69, 13, 179, 218]

In [ ]:
# Tokenize docs
taxonomy_description = taxonomy['description'].tolist()
taxonomy_name = taxonomy['name'].tolist()

tokenized_taxonomy = []
for i in range(len(taxonomy_name)):
    tokenized_taxonomy.append(preprocessing(taxonomy_name[i] + ': ' + taxonomy_description[i]))

# Tokenize queries
tokenized_infoSheet = []
for each in search_term_indices:
    if infoSheet['taxonomy heading ids'][each] != '0':
        tokenized_infoSheet.append(preprocessing(infoSheet['name_en'][each] + ': ' + infoSheet['abstract_en'][each]))

In [ ]:
bm25 = BM25Okapi(tokenized_taxonomy)

topK = 100

bm25_score = []
for query in tokenized_infoSheet:
    scores = bm25.get_scores(query)
    top_results = torch.topk(torch.tensor(scores), topK)
    bm25_score.append(top_results)

In [ ]:
bm25_score[0]

torch.return_types.topk(
values=tensor([32.4479, 29.7269, 28.3914, 24.1276, 23.1351, 22.1584, 20.6392, 18.4970,
        17.1057, 17.0532, 16.9368, 15.7475, 15.3574, 15.0684, 14.8499, 14.6540,
        14.3522, 13.8719, 13.4870, 13.4765, 13.3385, 13.1772, 13.0198, 12.4854,
        11.9184, 11.7834, 11.7180, 11.5898, 11.0874, 11.0570, 10.6394, 10.5587,
        10.5230, 10.5055, 10.4071, 10.2259, 10.0514,  9.9997,  9.8528,  9.7641,
         9.7146,  9.6927,  9.6278,  9.5878,  9.5819,  9.4510,  9.4044,  9.3553,
         9.3336,  9.2563,  9.2559,  9.2521,  9.2155,  9.2055,  9.1330,  9.1217,
         9.1004,  8.9923,  8.9191,  8.8344,  8.8288,  8.7691,  8.6995,  8.6939,
         8.6695,  8.6210,  8.5371,  8.4876,  8.4376,  8.3500,  8.2089,  7.9582,
         7.8913,  7.8843,  7.8111,  7.7545,  7.7441,  7.7208,  7.7183,  7.7012,
         7.6446,  7.5894,  7.5814,  7.5708,  7.5173,  7.5163,  7.4347,  7.4212,
         7.4153,  7.3627,  7.3595,  7.3457,  7.2985,  7.2689,  7.2620,  7.2326,
        

## OpenAI Search

Re-rank top 100 results retrivaled by BM25

In [ ]:
def takeFirst(elem):
    return elem[0]

def search(query_indices, doc_indices, topK=10):
    if topK > len(taxonomy):
        topK = len(taxonomy)

    result = []

    for i in range((len(query_indices))):
        query_embedding = infoSheet['embedding'][query_indices[i]]
        tmp = []
        for j in doc_indices[i]:
            doc_embedding = taxonomy['embedding'][j]
            tmp.append((cosine_similarity(doc_embedding, query_embedding), j))
        tmp.sort(key=takeFirst, reverse=True)
        result.append(tmp[:10])

    return result

doc_indices = []
for each in bm25_score:
    doc_indices.append(each[1].tolist())

result = search(search_term_indices, doc_indices)

In [ ]:
predictions = {'infoSheet_index': [], 'infoSheet_name': [], 'pred_taxonomy_index': [], 'pred_taxonomy_name': [], 'score': [], 'gold_taxonomy_index': []}
for q in range(len(result)):
    for (score, idx) in result[q]:
        predictions['infoSheet_index'].append(search_term_indices[q])
        predictions['infoSheet_name'].append(infoSheet['name_en'][search_term_indices[q]])
        predictions['pred_taxonomy_index'].append(idx+1)
        predictions['pred_taxonomy_name'].append(taxonomy['name'][idx])
        predictions['score'].append(score)
        predictions['gold_taxonomy_index'].append(infoSheet['taxonomy heading ids'][search_term_indices[q]].split(','))

predictions = pd.DataFrame(predictions)
predictions.head(10)

,infoSheet_index,infoSheet_name,pred_taxonomy_index,pred_taxonomy_name,score,gold_taxonomy_index
0,113,Learning Disabilities,25,Learning Disorders,0.907325,"[0, 25, 98, 220, 36]"
1,113,Learning Disabilities,110,Asperger Syndrome,0.894902,"[0, 25, 98, 220, 36]"
2,113,Learning Disabilities,21,"Developmental, Intellectual Delay and Disabili...",0.880416,"[0, 25, 98, 220, 36]"
3,113,Learning Disabilities,13,Attention Deficit Disorders (ADD/ADHD),0.868375,"[0, 25, 98, 220, 36]"
4,113,Learning Disabilities,57,Physical Disabilities,0.865600,"[0, 25, 98, 220, 36]"
5,113,Learning Disabilities,16,Behaviour and Conduct Problems,0.860598,"[0, 25, 98, 220, 36]"
6,113,Learning Disabilities,48,Obsessive Compulsive Disorder (OCD),0.853025,"[0, 25, 98, 220, 36]"
7,113,Learning Disabilities,46,Auditory Processing Disorder (APD),0.851302,"[0, 25, 98, 220, 36]"
8,113,Learning Disabilities,15,Autism and Autism Spectrum Disorders,0.849179,"[0, 25, 98, 220, 36]"
9,113,Learning Disabilities,167,Somatoform Disorders,0.839468,"[0, 25, 98, 220, 36]"


In [ ]:
predictions.to_csv(DATA_PATH + 'data/predictions/bm25-openai.csv', index=False)